#Preprocessing MP4 videos into Dataset

"""MP4 croping"""

In [ ]:
import os
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose

# -------------------
# Step 1: Get bounding box for one frame
# -------------------
def get_person_bbox(frame, pose_model):
    results = pose_model.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if not results.pose_landmarks:
        return None
    
    h, w, _ = frame.shape
    xs = [lm.x * w for lm in results.pose_landmarks.landmark]
    ys = [lm.y * h for lm in results.pose_landmarks.landmark]
    
    x_min, x_max = int(min(xs)), int(max(xs))
    y_min, y_max = int(min(ys)), int(max(ys))
    
    return x_min, y_min, x_max, y_max

# -------------------
# Step 2: Find bounding box for whole video
# -------------------
def get_video_bbox(video_path):
    cap = cv2.VideoCapture(video_path)
    with mp_pose.Pose(static_image_mode=False) as pose:
        x_min_total, y_min_total = 1e9, 1e9
        x_max_total, y_max_total = 0, 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            bbox = get_person_bbox(frame, pose)
            if bbox:
                x_min, y_min, x_max, y_max = bbox
                x_min_total = min(x_min_total, x_min)
                y_min_total = min(y_min_total, y_min)
                x_max_total = max(x_max_total, x_max)
                y_max_total = max(y_max_total, y_max)
    
    cap.release()
    if x_max_total == 0 and y_max_total == 0:
        return None
    return x_min_total, y_min_total, x_max_total, y_max_total

# -------------------
# Step 3: Crop with fixed square size
# -------------------
def crop_person_from_video(input_path, output_path, output_size=256):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    bbox = get_video_bbox(input_path)
    if not bbox:
        print(f"No person detected in {input_path}")
        return
    x_min, y_min, x_max, y_max = bbox
    
    # Add padding
    x_min = max(0, x_min-20)
    y_min = max(0, y_min-20)
    x_max = int(x_max+20)
    y_max = int(y_max+20)

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    x_max = min(w, x_max)
    y_max = min(h, y_max)

    # Make it square
    box_w = x_max - x_min
    box_h = y_max - y_min
    side = max(box_w, box_h)
    cx = (x_min + x_max) // 2
    cy = (y_min + y_max) // 2
    x_min = max(0, cx - side // 2)
    x_max = min(w, cx + side // 2)
    y_min = max(0, cy - side // 2)
    y_max = min(h, cy + side // 2)

    out = cv2.VideoWriter(output_path, fourcc, fps, (output_size, output_size))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cropped = frame[y_min:y_max, x_min:x_max]
        resized = cv2.resize(cropped, (output_size, output_size))
        out.write(resized)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# -------------------
# Step 4: Process all videos in folder
# -------------------
def crop_videos_in_folder(input_folder, output_folder, output_size=256):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".mp4"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            print(f"Processing {input_path} -> {output_path}")
            crop_person_from_video(input_path, output_path, output_size)

# -------------------
# 🚀 Run for your LOCK folder
# -------------------
input_folder = r"D:\Dhanvanth\SL\test"
output_folder = r"D:\Dhanvanth\SL\videos_cropped1\test"

# All outputs will be 256x256 square videos
crop_videos_in_folder(input_folder, output_folder, output_size=256)


Processing D:\Dhanvanth\SL\test\W23P55.mp4 -> D:\Dhanvanth\SL\videos_cropped1\test\W23P55.mp4


"""Croped MP4 to adding pose to it"""

In [24]:
import cv2
import mediapipe as mp
import os

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# -------------------
# Draw pose on a video
# -------------------
def draw_pose_on_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convert to RGB
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            # Draw pose if detected
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame, 
                    results.pose_landmarks, 
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),  # keypoints
                    mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)   # skeleton
                )
            
            out.write(frame)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Pose overlay saved -> {output_path}")

# -------------------
# Process all cropped videos in a folder
# -------------------
def add_pose_to_videos(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".mp4"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            print(f"Processing {input_path} -> {output_path}")
            draw_pose_on_video(input_path, output_path)

# -------------------
# 🚀 Run for your LOCK cropped videos
# -------------------
input_cropped_folder = r"D:\Dhanvanth\SL\videos_cropped1\test"
output_pose_folder   = r"D:\Dhanvanth\SL\videos_pose1\test"

add_pose_to_videos(input_cropped_folder, output_pose_folder)


Processing D:\Dhanvanth\SL\videos_cropped1\test\W23P55.mp4 -> D:\Dhanvanth\SL\videos_pose1\test\W23P55.mp4
Pose overlay saved -> D:\Dhanvanth\SL\videos_pose1\test\W23P55.mp4


"""holistic pose"""

In [ ]:
import cv2
import mediapipe as mp
import os

# ✅ Use Holistic instead of only Pose
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# -------------------
# Draw holistic landmarks on a video
# -------------------
def draw_holistic_on_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as holistic:

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Convert to RGB
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(rgb)

            # -------------------
            # Draw Pose (33 landmarks)
            # -------------------
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
                    mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1)
                )

            # -------------------
            # Draw Left & Right Hands (21 + 21 landmarks)
            # -------------------
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1),
                    mp_drawing.DrawingSpec(color=(0,255,255), thickness=1, circle_radius=1)
                )
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1),
                    mp_drawing.DrawingSpec(color=(0,255,255), thickness=1, circle_radius=1)
                )

            # -------------------
            # (Optional) Draw Face (468 landmarks – can be disabled)
            # -------------------
            # if results.face_landmarks:
            #     mp_drawing.draw_landmarks(
            #         frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            #         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            #         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
            #     )

            out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Holistic pose overlay saved -> {output_path}")

# -------------------
# Process all cropped videos in a folder
# -------------------
def add_holistic_to_videos(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".mp4"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            print(f"Processing {input_path} -> {output_path}")
            draw_holistic_on_video(input_path, output_path)

# -------------------
# 🚀 Run for your test cropped videos
# -------------------
input_cropped_folder = r"D:\Dhanvanth\SL\videos_cropped1\test"
output_pose_folder   = r"D:\Dhanvanth\SL\videos_pose1\test"

add_holistic_to_videos(input_cropped_folder, output_pose_folder)


Processing D:\Dhanvanth\SL\videos_cropped1\test\W23P55.mp4 -> D:\Dhanvanth\SL\videos_pose1\test\W23P55.mp4
Holistic pose overlay saved -> D:\Dhanvanth\SL\videos_pose1\test\W23P55.mp4


"""pose to npz"""

In [17]:
import os
import numpy as np
import cv2
import mediapipe as mp

mp_holistic = mp.solutions.holistic

# -------------------
# Function to extract pose/hand keypoints from one video
# -------------------
def extract_pose_to_npz(video_path, out_npz_path, include_face=False, output_size=None):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    pixel_coords = []  # list of (K,2)
    masks = []

    with mp_holistic.Holistic(static_image_mode=False,
                              min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            h, w = frame.shape[:2]
            if output_size is not None:
                frame = cv2.resize(frame, (output_size, output_size))
                h, w = output_size, output_size
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(rgb)

            # initialize all K to nan and mask False
            K = 33 + 21 + 21
            coords = np.full((K, 2), np.nan, dtype=np.float32)
            mask = np.zeros((K,), dtype=np.uint8)

            # pose (33)
            if results.pose_landmarks:
                for i, lm in enumerate(results.pose_landmarks.landmark):
                    coords[i, 0] = lm.x * w
                    coords[i, 1] = lm.y * h
                    mask[i] = 1

            # left hand (21) -> indices 33..53
            if results.left_hand_landmarks:
                for i, lm in enumerate(results.left_hand_landmarks.landmark):
                    coords[33 + i, 0] = lm.x * w
                    coords[33 + i, 1] = lm.y * h
                    mask[33 + i] = 1

            # right hand (21) -> indices 54..74
            if results.right_hand_landmarks:
                for i, lm in enumerate(results.right_hand_landmarks.landmark):
                    coords[54 + i, 0] = lm.x * w
                    coords[54 + i, 1] = lm.y * h
                    mask[54 + i] = 1

            pixel_coords.append(coords)
            masks.append(mask)

    cap.release()
    pixel_coords = np.stack(pixel_coords, axis=0)   # (T, K, 2)
    masks = np.stack(masks, axis=0).astype(np.uint8) # (T, K)

    # Normalization
    T, K, _ = pixel_coords.shape
    norm = np.zeros_like(pixel_coords, dtype=np.float32)
    for t in range(T):
        pts = pixel_coords[t]  # (K,2)
        mask = masks[t].astype(bool)

        # origin = mid-hip (23,24) if available
        origin = None
        if mask[23] and mask[24]:
            origin = (pts[23] + pts[24]) / 2.0
        else:
            visible_pose_idx = np.where(mask[:33])[0]
            if len(visible_pose_idx) > 0:
                origin = np.nanmean(pts[visible_pose_idx], axis=0)
        if origin is None:
            origin = np.array([0.0, 0.0], dtype=np.float32)

        # scale = shoulder distance (11,12) else bbox diag
        if mask[11] and mask[12]:
            scale = np.linalg.norm(pts[11] - pts[12])
            if scale < 1e-3:
                scale = 1.0
        else:
            vis_pts = pts[mask.astype(bool)]
            if vis_pts.size == 0:
                scale = 1.0
            else:
                bbox = np.nanmin(vis_pts, axis=0), np.nanmax(vis_pts, axis=0)
                diag = np.linalg.norm(bbox[1] - bbox[0])
                scale = max(diag, 1.0)

        norm[t] = (np.nan_to_num(pts) - origin[None, :]) / float(scale)

    # save
    np.savez_compressed(out_npz_path,
                        xy_pixels=pixel_coords.astype(np.float32),
                        xy_norm=norm.astype(np.float32),
                        mask=masks,
                        fps=float(fps))
    print("Saved:", out_npz_path)
    return out_npz_path


# -------------------
# Batch processing: convert all videos in folder
# -------------------
def convert_videos_to_npz(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".mp4"):
            video_path = os.path.join(input_folder, file_name)
            npz_name = os.path.splitext(file_name)[0] + ".npz"
            out_path = os.path.join(output_folder, npz_name)
            print(f"Processing {video_path} -> {out_path}")
            extract_pose_to_npz(video_path, out_path)


# -------------------
# 🚀 Run for your dataset
# -------------------
input_pose_folder = r"D:\Dhanvanth\SL\videos_pose1\test"
output_npy_folder = r"D:\Dhanvanth\SL\pose_to_npy\test"

convert_videos_to_npz(input_pose_folder, output_npy_folder)


Processing D:\Dhanvanth\SL\videos_pose1\test\W23P55.mp4 -> D:\Dhanvanth\SL\pose_to_npy\test\W23P55.npz
Saved: D:\Dhanvanth\SL\pose_to_npy\test\W23P55.npz


"""holistic pose"""

In [23]:
import os
import cv2
import numpy as np
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# -------------------
# Draw holistic pose/hand landmarks on one video
# -------------------
def draw_pose_on_video_holistic(input_path, output_path, skeleton_only=False, output_size=None):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if output_size:
        out_w, out_h = output_size, output_size
    else:
        out_w, out_h = w, h

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, float(fps), (out_w, out_h))

    with mp_holistic.Holistic(static_image_mode=False,
                              min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if output_size:
                frame_proc = cv2.resize(frame, (output_size, output_size))
            else:
                frame_proc = frame.copy()

            rgb = cv2.cvtColor(frame_proc, cv2.COLOR_BGR2RGB)
            results = holistic.process(rgb)

            # Skeleton-only → black canvas, else original frame
            canvas = np.zeros_like(frame_proc) if skeleton_only else frame_proc

            # Pose (33 landmarks)
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    canvas,
                    results.pose_landmarks,
                    mp_holistic.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)
                )

            # Left hand (21 landmarks)
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(
                    canvas,
                    results.left_hand_landmarks,
                    mp_holistic.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0,255,255), thickness=2)
                )

            # Right hand (21 landmarks)
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(
                    canvas,
                    results.right_hand_landmarks,
                    mp_holistic.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0,255,255), thickness=2)
                )

            out.write(canvas)

    cap.release()
    out.release()
    print("Pose overlay saved ->", output_path)


# -------------------
# Batch process: all videos in a folder
# -------------------
def add_holistic_pose_to_videos(input_folder, output_folder,
                                skeleton_only=False, output_size=None):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".mp4"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            print(f"Processing {input_path} -> {output_path}")
            draw_pose_on_video_holistic(input_path, output_path,
                                        skeleton_only=skeleton_only,
                                        output_size=output_size)


# -------------------
# 🚀 Run for your dataset
# -------------------
input_videos = r"D:\Dhanvanth\SL\videos_cropped1\test"
output_videos = r"D:\Dhanvanth\SL\videos_pose_holistic\test"

# Example 1: Overlay on original video
# add_holistic_pose_to_videos(input_videos, output_videos,
#                             skeleton_only=False, output_size=256)

# Example 2: Skeleton-only version
add_holistic_pose_to_videos(input_videos, output_videos,
                            skeleton_only=True, output_size=256)


Processing D:\Dhanvanth\SL\videos_cropped1\test\W23P55.mp4 -> D:\Dhanvanth\SL\videos_pose_holistic\test\W23P55.mp4
Pose overlay saved -> D:\Dhanvanth\SL\videos_pose_holistic\test\W23P55.mp4


In [21]:
import numpy as np

def resample_sequence(xy, mask, target_len):
    # xy: (T, K, 2), mask: (T, K)
    T, K, _ = xy.shape
    if T == target_len:
        return xy, mask
    # handle NaNs: replace NaN with linear interpolation per joint
    xy_filled = xy.copy()
    for k in range(K):
        for dim in range(2):
            series = xy_filled[:, k, dim]
            valid = ~np.isnan(series)
            if valid.sum() == 0:
                xy_filled[:, k, dim] = 0.0
            elif valid.sum() == 1:
                xy_filled[:, k, dim] = series[valid][0]
            else:
                xp = np.where(valid)[0]
                fp = series[valid]
                x = np.arange(T)
                xy_filled[:, k, dim] = np.interp(x, xp, fp)
    # now interpolate in time to target_len
    old_idx = np.linspace(0, T-1, T)
    new_idx = np.linspace(0, T-1, target_len)
    out = np.zeros((target_len, K, 2), dtype=xy.dtype)
    out_mask = np.zeros((target_len, K), dtype=mask.dtype)
    for k in range(K):
        for d in range(2):
            out[:, k, d] = np.interp(new_idx, old_idx, xy_filled[:, k, d])
        # mask: mark visible if majority of nearby frames were visible
        vis = mask[:, k].astype(np.float32)
        out_mask[:, k] = np.interp(new_idx, old_idx, vis) > 0.5
    return out, out_mask
